In [ ]:
import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime, timedelta

In [ ]:
fake = Faker()

# Configuration
NUM_HIST = 1_000_000  # historical transactions
NUM_DAILY = 10_000   # daily transactions per day
DAYS = 5             # number of days to simulate

In [ ]:
# ฟังก์ชันสร้างข้อมูลรายการเดียว
def generate_transaction(idx, date):
    return {
        'transaction_id': f'TX{idx:07d}',
        'user_id': fake.random_int(min=1, max=100000),
        'product_id': fake.random_int(min=1, max=10000),
        'category': fake.word(ext_word_list=['electronics', 'clothing', 'home', 'toys']),
        'amount': round(np.random.uniform(10, 500), 2),
        'payment_method': fake.word(ext_word_list=['credit_card', 'paypal', 'bank_transfer']),
        'timestamp': date.strftime('%Y-%m-%d %H:%M:%S'),
        'status': 'completed'
    }

In [ ]:
# ฟังก์ชันเพิ่มปัญหาคุณภาพข้อมูล
def add_issues(df):
    # เพิ่มสถานะ void/cancel โดยไม่มีรายการต้นฉบับ
    void_count = int(len(df) * 0.005)
    df.loc[df.sample(void_count).index, 'status'] = np.random.choice(['void', 'cancel'], void_count)

    # เพิ่มยอดเงินติดลบ
    neg_count = int(len(df) * 0.002)
    df.loc[df.sample(neg_count).index, 'amount'] *= -1
    return df

In [ ]:
# สร้างข้อมูลย้อนหลัง
hist_data = [generate_transaction(i, fake.date_time_between(start_date='-1y', end_date='now')) for i in range(NUM_HIST)]
df_hist = pd.DataFrame(hist_data)
df_hist = add_issues(df_hist)
df_hist.to_parquet('historical_transactions.parquet', index=False)
print("สร้าง historical_transactions.parquet เรียบร้อยแล้ว")

In [ ]:
# สร้างข้อมูลรายวัน
base_idx = NUM_HIST
for day in range(DAYS):
    date = datetime.now() + timedelta(days=day)
    daily_data = [generate_transaction(base_idx + i, date) for i in range(NUM_DAILY)]
    df_day = pd.DataFrame(daily_data)
    df_day = add_issues(df_day)
    df_day.to_parquet(f'daily_transactions_day{day+1}.parquet', index=False)
    print(f"สร้าง daily_transactions_day{day+1}.parquet เรียบร้อยแล้ว")
